In [57]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
from datetime import datetime
import os
import pandas as pd
from random import randint
from bs4 import BeautifulSoup as bs
import requests
import numpy as np

In [6]:
begin_url = 'http://i.pgatour.com/image/upload/c_fill,h_120/holes_2017_r_002_'
end_url = '_stracka_green_1.png'

for i in range(101,800):
    url = begin_url + str(i) + end_url
    time.sleep(1)
    response = requests.get(url)
    
    if response.status_code != 404:
        print(i, response.status_code)
        
        
    



202 200
233 200
704 200


In [158]:
def split_combine(x):
    temp = x.split(', ')
    return temp[1] + ' ' + temp[0]

def replace_mi(x):
    periods = x.count('.')
    if periods == 1:
        name_ls = x.split(' ')
        if '.' in name_ls[1]:
            return name_ls[0] + ' ' + name_ls[2]
        else:
            return x
    else:
        return x


player_list = pd.read_csv('/home/valesco/Datasets/PGA_Data/player_id.csv')
player_list['player_query'] = player_list['player_name'].apply(lambda x: 
        split_combine(x))
player_list['player_query'] = player_list['player_query'].apply(lambda x: replace_mi(x))
player_list.head()

player_ls = player_list['player_query'].values
player_id_ls = player_list['player_id'].values
player_ls[0]

'Merrick  Francis'

In [155]:
player_rank_df = pd.DataFrame(columns = ['player_id', 'player_name', 'tournament_name', 'tournament_start_date',
         'tournament_end_date', 'num_rounds_played', 'first_round_score', 'second_round_score', 
         'third_round_score', 'fourth_round_score', 'total_score', 'score_vs_par', 'final_rank', 'earnings'])

skipped_player_df = pd.DataFrame(columns = ['player_id', 'player_name'])

count = 0
skipped_count = 0

mon_ls = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

url = 'www.fanooz.com'
login_name = 'valesco'
pword = '75799975'

player_url = 'http://fanooz.com/account/player_admin.php'
browser = webdriver.Chrome()
browser.get(player_url)

login = browser.find_element_by_xpath('/html/body/table[2]/tbody/tr[1]/td[2]/table/tbody/tr/td[1]/table/tbody/tr[6]/td[2]/form/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]/input')
login.send_keys(login_name + Keys.TAB + pword + Keys.ENTER)

time.sleep(2)
player_ls = ['Jon Rahm', 'Danny Lee']

for p_num, player_name in enumerate(player_ls):
    browser.get(player_url)
    player = player_name
    player_id = player_id_ls[p_num]
    #try:
    time.sleep(2)
    search_box = browser.find_element_by_xpath('//*[@name="filter_name"]')
    search_box.clear()
    time.sleep(1)
    search_box.send_keys(player + Keys.ENTER)
    time.sleep(3)

    #try:
    view_results = browser.find_element_by_xpath('//a[contains(@href, "players/profile.php?id")]')
    view_results.click()

    time.sleep(3)

    tourny_results = browser.find_elements_by_xpath('//tr[@class="tablebody"]')
    career_earnings = 0
    for result in tourny_results:
        if 'Worldwide Career' in result.text:
            career_earnings = result.text.split('$')[1].replace(',', '').replace('*', '')

    table_body = browser.find_elements_by_xpath('//tr[@class="tablebody"]')

    href = browser.find_element_by_xpath('//a[contains(@href, "&type=results")]')
    href.click()

    time.sleep(2) 

    number_of_tournies = browser.find_element_by_xpath('//*[@name="filter_num_tournaments"]/option[@value="500"]')
    number_of_tournies.click()

    time.sleep(1)
    set_filter = browser.find_element_by_xpath('//*[@name="setfilter"]')
    set_filter.click()

    time.sleep(3)

    tables = browser.find_elements_by_xpath('//table[@class="tableborder"]')
    
    table_elements = tables[3].find_elements_by_xpath('//tr[@class="tablebody"]')


    for e, element in enumerate(table_elements):
        try:
            td = element.find_elements_by_tag_name('td')
            row_ls = []
            for cell in td:
                row_ls.append(cell.text)

            if len(row_ls) > 8:
                if row_ls[1] == 'S  O':
                    temp = row_ls[2].split('\n')
                    tournament_name = temp[0]
                    dates = temp[1]            
                    dash_ls = dates.split('-')
                    start_mon = dash_ls[0].split(' ')[0]
                    start_day = dash_ls[0].split(' ')[1]
                    end_day = dash_ls[1].split(',')[0]
                    year = dash_ls[1].split(',')[1]

                    end_mon = start_mon
                    for mon in mon_ls:
                        if mon in end_day:
                            end_month = mon
                            end_day = end_day.replace(mon, '')  

                    for var in ['start_mon', 'start_day', 'year', 'end_mon', 'end_day']:
                        globals()[var] = globals()[var].replace(' ', '')

                    round_1_score = row_ls[3]
                    round_2_score = row_ls[4]
                    round_3_score = row_ls[5]
                    round_4_score = row_ls[6]
                    total_score = row_ls[-4]
                    score_to_par = row_ls[-3]
                    tournament_rank = row_ls[-2]
                    earnings = row_ls[-1].replace('$', '').replace(',','')

                    num_rounds_played = 0

                    for var in ['round_1_score', 'round_2_score', 'round_3_score', 'round_4_score', 'earnings']:
                        try:
                            int(globals()[var])
                            if var != 'earnings':
                                num_rounds_played += 1
                        except:
                            globals()[var] = np.nan
                            pass


                    start_date_str = '{} {} {}'.format(start_mon, start_day, year)
                    end_date_str = '{} {} {}'.format(end_mon, end_day, year)
                    start_date = datetime.strptime(start_date_str, '%b %d %Y')
                    end_date = datetime.strptime(end_date_str, '%b %d %Y')

                    player_rank_df.loc[count] = [player_id, player, tournament_name, start_date, end_date, 
                                num_rounds_played, round_1_score, round_2_score, round_3_score, round_4_score,
                                total_score, score_to_par, tournament_rank, earnings]
                    print(count)
                    count += 1
        except:
            print(tournament_name)
#except:
    #skipped_player_df.loc[skipped_count] = [player_id, player_name]
    #skipped_count += 1

8
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
9
Puerto Rico Open
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253


In [160]:
len(player_ls[540:])

520

In [154]:
row_ls[-1]

''

In [82]:
temp_end =  dash_ls[1].split(',')[0]


for mon in mon_ls:
    if mon in temp_end:
        print(mon)
        end_month = mon
        end_day = temp_end.replace(mon, '')

end_month, end_day

Apr


('Apr', '  3')